# Lab 2: Fine-tuning AfroXLMR for African Language Sentiment Analysis

## Learning Objectives
In this lab, you will:
1. Load a pre-trained multilingual model (AfroXLMR)
2. Understand the fine-tuning process
3. Fine-tune AfroXLMR on AfriSenti dataset for sentiment analysis
4. Evaluate model performance
5. Compare results across different African languages

## Background
**AfroXLMR** is an African-adapted XLM-RoBERTa model pre-trained on 16 African languages. It's an encoder model (like BERT) that excels at understanding tasks.

**AfriSenti** is a sentiment analysis dataset covering 14 African languages with tweets labeled as positive, negative, or neutral.

## Why Fine-tuning?
Remember from the lecture: Pre-training teaches general language knowledge (like going to school), while fine-tuning specializes the model for a specific task (like medical school).

## Step 1: Install Required Libraries

In [ ]:
!pip install transformers datasets torch accelerate scikit-learn seaborn -q

## Step 2: Import Libraries

In [ ]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
from sklearn.metrics import (
    accuracy_score,
    precision_recall_fscore_support,
    confusion_matrix,
    classification_report
)
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore')

# Set random seeds
torch.manual_seed(42)
np.random.seed(42)

# Check device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
if device == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## Step 3: Choose Your Language

AfriSenti supports 14 African languages. Choose one to work with!

Available languages:
- **amh**: Amharic
- **arq**: Algerian Arabic
- **hau**: Hausa
- **ibo**: Igbo
- **kin**: Kinyarwanda
- **ary**: Moroccan Arabic
- **pcm**: Nigerian Pidgin
- **oro**: Oromo
- **por**: Portuguese (Mozambican)
- **swa**: Swahili
- **tir**: Tigrinya
- **twi**: Twi
- **tso**: Xitsonga
- **yor**: Yoruba

In [ ]:
LANGUAGE = "hau"  # Change this to any language code from the list above

# Language names for display
LANGUAGE_NAMES = {
    "amh": "Amharic",
    "arq": "Algerian Arabic",
    "hau": "Hausa",
    "ibo": "Igbo",
    "kin": "Kinyarwanda",
    "ary": "Moroccan Arabic",
    "pcm": "Nigerian Pidgin",
    "oro": "Oromo",
    "por": "Portuguese (Mozambican)",
    "swa": "Swahili",
    "tir": "Tigrinya",
    "twi": "Twi",
    "tso": "Xitsonga",
    "yor": "Yoruba"
}

print(f"\n{'='*60}")
print(f"Selected Language: {LANGUAGE_NAMES[LANGUAGE]} ({LANGUAGE})")
print(f"{'='*60}\n")

## Step 4: Load the AfriSenti Dataset

We'll load the dataset for your chosen language from Hugging Face.

In [ ]:
# Load AfriSenti-Twitter dataset directly from GitHub
# Note: HuggingFace no longer supports dataset scripts, so we load directly from the repository
print(f"Loading AfriSenti dataset for {LANGUAGE_NAMES[LANGUAGE]}...")

BASE_URL = f"https://raw.githubusercontent.com/afrisenti-semeval/afrisent-semeval-2023/main/data/{LANGUAGE}"

# Load the data using pandas
train_df = pd.read_csv(f"{BASE_URL}/train.tsv", sep='\t')
val_df = pd.read_csv(f"{BASE_URL}/dev.tsv", sep='\t')
test_df = pd.read_csv(f"{BASE_URL}/test.tsv", sep='\t')

label_to_id = {"negative": 0, "neutral": 1, "positive": 2}
id_to_label = {v: k for k, v in label_to_id.items()}

train_df["label"] = train_df["label"].map(label_to_id)
val_df["label"]   = val_df["label"].map(label_to_id)
test_df["label"]  = test_df["label"].map(label_to_id)

print("\nDataset loaded successfully!")
print(f"\nTraining set shape: {train_df.shape}")
print(f"Validation set shape: {val_df.shape}")
print(f"Test set shape: {test_df.shape}")
print("\nColumns:", train_df.columns.tolist())

# Show the first few examples
print(f"\nFirst 3 examples from training set:")
print("="*80)

for i in range(3):
    row = train_df.iloc[i]
    print(f"\nExample {i+1}:")
    print(f"Text: {row['tweet']}")
    print(f"Label ID: {row['label']} ({id_to_label[row['label']]})")
    print("-"*80)

## Step 5: Explore the Dataset

Let's understand the data distribution.

In [ ]:
# Dataset statistics
print(f"\nDataset Statistics for {LANGUAGE_NAMES[LANGUAGE]}:")
print("="*60)
print(f"Training samples:   {len(train_df):,}")
print(f"Validation samples: {len(val_df):,}")
print(f"Test samples:       {len(test_df):,}")
print(f"Total samples:      {len(train_df) + len(val_df) + len(test_df):,}")

# Label distribution
train_dist = train_df['label'].value_counts().to_dict()
valid_dist = val_df['label'].value_counts().to_dict()
test_dist = test_df['label'].value_counts().to_dict()

print("\nLabel Distribution:")
print("="*60)
print(f"{'Split':<12} {'Negative':<12} {'Neutral':<12} {'Positive':<12}")
print("-"*60)

NEG, NEU, POS = 0, 1, 2

print(f"{'Train':<12} {train_dist.get(NEG, 0):<12} {train_dist.get(NEU, 0):<12} {train_dist.get(POS, 0):<12}")
print(f"{'Validation':<12} {valid_dist.get(NEG, 0):<12} {valid_dist.get(NEU, 0):<12} {valid_dist.get(POS, 0):<12}")
print(f"{'Test':<12} {test_dist.get(NEG, 0):<12} {test_dist.get(NEU, 0):<12} {test_dist.get(POS, 0):<12}")

label_names = {0: "negative", 1: "neutral", 2: "positive"}

# Visualize distribution
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for idx, (split_name, dist) in enumerate([('Train', train_dist), ('Validation', valid_dist), ('Test', test_dist)]):
    labels_list = [label_names[k] for k in sorted(dist.keys())]
    values = [dist[k] for k in sorted(dist.keys())]

    axes[idx].bar(labels_list, values, color=['#e74c3c', '#95a5a6', '#2ecc71'])
    axes[idx].set_title(f'{split_name} Set')
    axes[idx].set_ylabel('Count')
    axes[idx].set_xlabel('Sentiment')

    # Add value labels on bars
    for i, v in enumerate(values):
        axes[idx].text(i, v + max(values)*0.02, str(v), ha='center', va='bottom')

plt.suptitle(f'Sentiment Distribution - {LANGUAGE_NAMES[LANGUAGE]}', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig(f'afrisenti_{LANGUAGE}_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

# Check for class imbalance
total_train = sum(train_dist.values())
imbalance_ratio = max(train_dist.values()) / min(train_dist.values())
print(f"\nClass Imbalance Ratio: {imbalance_ratio:.2f}")
if imbalance_ratio > 3:
    print("⚠️  Significant class imbalance detected. Consider using class weights!")
else:
    print("✓ Class distribution is relatively balanced.")

## Step 6: Load Pre-trained AfroXLMR Model

AfroXLMR was pre-trained on 16 African languages. We'll fine-tune it for sentiment analysis.

In [ ]:
# Model name on Hugging Face
MODEL_NAME = "Davlan/afro-xlmr-small"

print(f"Loading AfroXLMR model and tokenizer...")
print(f"Model: {MODEL_NAME}\n")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Load model for sequence classification (sentiment analysis)
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=3,  # 3 classes: negative, neutral, positive
    problem_type="single_label_classification"
)

model = model.to(device)

print(f"✓ Model loaded successfully!")
print(f"✓ Tokenizer loaded successfully!")
print(f"\nModel has {model.num_parameters():,} parameters")
print(f"Model is on device: {next(model.parameters()).device}")

## Step 7: Tokenize the Dataset

We need to convert text into tokens that the model can understand.

In [ ]:
# Convert pandas DataFrames to Hugging Face datasets
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(
        examples['tweet'],
        padding=False,  # We'll pad dynamically in batches
        truncation=True,
        max_length=128,  # Maximum sequence length
    )

# Tokenize all splits
print("Tokenizing datasets...")
print("This may take a minute...\n")

tokenized_train = train_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=['tweet']  # Remove text column, keep only tokens and labels
)

tokenized_valid = val_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=['tweet']
)

tokenized_test = test_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=['tweet']
)

print("✓ Tokenization complete!\n")

# Example of tokenization
example = train_df.iloc[0]
tokens = tokenizer.tokenize(example['tweet'])
token_ids = tokenizer.convert_tokens_to_ids(tokens)

print("Example of Tokenization:")
print("="*80)
print(f"Original text: {example['tweet']}")
print(f"\nTokens: {tokens[:10]}... ({len(tokens)} total)")
print(f"\nToken IDs: {token_ids[:10]}... ({len(token_ids)} total)")
print(f"\nLabel: {label_names[example['label']]}")

## Step 8: Set Up Data Collator

The data collator handles dynamic padding of sequences in each batch.

In [ ]:
# Data collator for dynamic padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

print("Data collator created!")
print("\nWhat does the data collator do?")
print("- Pads sequences to the same length within each batch")
print("- Saves memory by not padding all sequences to max length")
print("- Creates attention masks to tell the model which tokens are real vs. padding")

## Step 9: Define Evaluation Metrics

We'll compute accuracy, precision, recall, and F1-score.

In [ ]:
def compute_metrics(eval_pred):
    """Compute metrics for evaluation"""
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    # Calculate metrics
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average='weighted', zero_division=0
    )

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }

print("Evaluation metrics defined!")
print("\nMetrics we'll track:")
print("- Accuracy: Overall correctness")
print("- Precision: How many predicted positives are actually positive")
print("- Recall: How many actual positives we correctly identified")
print("- F1-score: Harmonic mean of precision and recall")

## Step 10: Configure Training Arguments

These settings control how the model is fine-tuned.

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir=f"afroXLMR_{LANGUAGE}_sentiment",

    # Training hyperparameters
    num_train_epochs=1,              # Number of times to go through the data
    per_device_train_batch_size=16,  # Batch size for training
    per_device_eval_batch_size=32,   # Batch size for evaluation
    learning_rate=2e-5,              # Learning rate (small for fine-tuning)
    weight_decay=0.01,               # Regularization to prevent overfitting
    warmup_ratio=0.1,                # Gradually increase learning rate

    # Evaluation and saving
    eval_strategy="steps",           # Evaluate during training
    eval_steps=200,                  # Evaluate every 100 steps
    save_strategy="steps",
    save_steps=200,
    save_total_limit=2,              # Keep only 2 best checkpoints
    load_best_model_at_end=True,     # Load best model after training
    metric_for_best_model="f1",      # Use F1-score to determine best model

    # Logging
    logging_dir=f"logs_{LANGUAGE}",
    logging_steps=50,
    report_to="none",                # Don't send to wandb/tensorboard

    # Other settings
    fp16=torch.cuda.is_available(),  # Use mixed precision on GPU
    dataloader_num_workers=2,
    seed=42,
)

print("Training configuration:")
print("="*60)
print(f"Number of epochs: {training_args.num_train_epochs}")
print(f"Batch size: {training_args.per_device_train_batch_size}")
print(f"Learning rate: {training_args.learning_rate}")
print(f"Evaluation strategy: Every {training_args.eval_steps} steps")
print(f"Mixed precision (FP16): {training_args.fp16}")

## Step 11: Create Trainer and Start Fine-tuning

Now we'll fine-tune the model! This is where the magic happens. 🎉

In [ ]:
# Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

print(f"\n{'='*80}")
print(f"Starting Fine-tuning for {LANGUAGE_NAMES[LANGUAGE]} Sentiment Analysis")
print(f"{'='*80}\n")
print("This will take several minutes. Please be patient...\n")

# Train the model
train_result = trainer.train()

print(f"\n{'='*80}")
print("Fine-tuning Complete! 🎉")
print(f"{'='*80}\n")

# Print training results
print("Training Results:")
print("-"*60)
print(f"Training Loss: {train_result.training_loss:.4f}")
print(f"Training Runtime: {train_result.metrics['train_runtime']:.2f} seconds")
print(f"Training Samples/Second: {train_result.metrics['train_samples_per_second']:.2f}")

## Step 12: Evaluate on Test Set

Let's see how well our fine-tuned model performs on unseen data!

In [ ]:
print(f"\nEvaluating on {LANGUAGE_NAMES[LANGUAGE]} test set...\n")

# Evaluate on test set
test_results = trainer.evaluate(tokenized_test)

print(f"\n{'='*80}")
print(f"Test Results for {LANGUAGE_NAMES[LANGUAGE]}")
print(f"{'='*80}")
print(f"Accuracy:  {test_results['eval_accuracy']:.4f} ({test_results['eval_accuracy']*100:.2f}%)")
print(f"Precision: {test_results['eval_precision']:.4f}")
print(f"Recall:    {test_results['eval_recall']:.4f}")
print(f"F1-Score:  {test_results['eval_f1']:.4f}")
print(f"{'='*80}\n")

## Step 13: Detailed Classification Report

Let's get per-class metrics to see how well the model performs on each sentiment.

In [ ]:
# Get predictions on test set
predictions = trainer.predict(tokenized_test)
predicted_labels = np.argmax(predictions.predictions, axis=1)
true_labels = predictions.label_ids

# Classification report
report = classification_report(
    true_labels,
    predicted_labels,
    target_names=['Negative', 'Neutral', 'Positive'],
    digits=4
)

print("\nDetailed Classification Report:")
print("="*80)
print(report)

# Save report
with open(f'classification_report_{LANGUAGE}.txt', 'w') as f:
    f.write(f"Classification Report for {LANGUAGE_NAMES[LANGUAGE]}\n")
    f.write("="*80 + "\n\n")
    f.write(report)

## Step 14: Confusion Matrix

A confusion matrix shows which classes the model confuses with each other.

In [ ]:
# Compute confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)

# Plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(
    cm,
    annot=True,
    fmt='d',
    cmap='Blues',
    xticklabels=['Negative', 'Neutral', 'Positive'],
    yticklabels=['Negative', 'Neutral', 'Positive'],
    cbar_kws={'label': 'Count'}
)
plt.title(f'Confusion Matrix - {LANGUAGE_NAMES[LANGUAGE]} Sentiment Analysis', fontsize=14, fontweight='bold')
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.tight_layout()
plt.savefig(f'confusion_matrix_{LANGUAGE}.png', dpi=150, bbox_inches='tight')
plt.show()

# Print confusion matrix analysis
print("\nConfusion Matrix Analysis:")
print("="*80)
print(f"Correctly classified Negative: {cm[0, 0]} out of {cm[0].sum()}")
print(f"Correctly classified Neutral:  {cm[1, 1]} out of {cm[1].sum()}")
print(f"Correctly classified Positive: {cm[2, 2]} out of {cm[2].sum()}")
print("\nCommon misclassifications:")
if cm[0, 1] > 5:
    print(f"  - {cm[0, 1]} Negative tweets classified as Neutral")
if cm[0, 2] > 5:
    print(f"  - {cm[0, 2]} Negative tweets classified as Positive")
if cm[2, 1] > 5:
    print(f"  - {cm[2, 1]} Positive tweets classified as Neutral")
if cm[2, 0] > 5:
    print(f"  - {cm[2, 0]} Positive tweets classified as Negative")

## Step 15: Test on Custom Examples

Now let's test the model on some custom examples!

In [ ]:
def predict_sentiment(text, model, tokenizer):
    """Predict sentiment for a given text"""
    # Tokenize
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=128)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Get prediction
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probabilities = torch.softmax(logits, dim=1)
        predicted_class = torch.argmax(probabilities, dim=1).item()

    label_names = {0: "Negative", 1: "Neutral", 2: "Positive"}

    return {
        'label': label_names[predicted_class],
        'confidence': probabilities[0][predicted_class].item(),
        'probabilities': {
            'Negative': probabilities[0][0].item(),
            'Neutral': probabilities[0][1].item(),
            'Positive': probabilities[0][2].item(),
        }
    }

# Example texts (you can modify these)
# These are generic examples - replace with language-specific examples!
custom_examples = [
    "Wannan abun yayi kyau, ina son shi!",
    "Wannan abun bai yi kyau ba. Na tsane shi!",
    "Yau ana ruwa a Cape Town",
]

print(f"\nTesting Custom Examples for {LANGUAGE_NAMES[LANGUAGE]}:")
print("="*80)

for i, example in enumerate(custom_examples, 1):
    result = predict_sentiment(example, model, tokenizer)

    print(f"\nExample {i}:")
    print(f"Text: {example}")
    print(f"Predicted: {result['label']} (Confidence: {result['confidence']:.2%})")
    print(f"Probabilities:")
    for sentiment, prob in result['probabilities'].items():
        print(f"  {sentiment:>8}: {prob:6.2%} {'█' * int(prob * 50)}")
    print("-"*80)

## Step 16: Visualize Training History

Let's plot the training and validation metrics over time.

In [ ]:
# Extract training history
log_history = trainer.state.log_history

# Separate training and evaluation logs
train_logs = [log for log in log_history if 'loss' in log]
eval_logs = [log for log in log_history if 'eval_loss' in log]

# Extract metrics
train_steps = [log['step'] for log in train_logs]
train_losses = [log['loss'] for log in train_logs]

eval_steps = [log['step'] for log in eval_logs]
eval_losses = [log['eval_loss'] for log in eval_logs]
eval_f1 = [log['eval_f1'] for log in eval_logs]
eval_accuracy = [log['eval_accuracy'] for log in eval_logs]

# Plot
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Loss
axes[0].plot(train_steps, train_losses, label='Training Loss', marker='o', markersize=4)
axes[0].plot(eval_steps, eval_losses, label='Validation Loss', marker='s', markersize=4)
axes[0].set_xlabel('Steps')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training and Validation Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# F1-Score
axes[1].plot(eval_steps, eval_f1, label='Validation F1', marker='o', color='green', markersize=4)
axes[1].set_xlabel('Steps')
axes[1].set_ylabel('F1-Score')
axes[1].set_title('Validation F1-Score')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Accuracy
axes[2].plot(eval_steps, eval_accuracy, label='Validation Accuracy', marker='s', color='purple', markersize=4)
axes[2].set_xlabel('Steps')
axes[2].set_ylabel('Accuracy')
axes[2].set_title('Validation Accuracy')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.suptitle(f'Training History - {LANGUAGE_NAMES[LANGUAGE]}', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig(f'training_history_{LANGUAGE}.png', dpi=150, bbox_inches='tight')
plt.show()

## Step 17: Save Your Fine-tuned Model

Save the model so you can use it later!

In [ ]:
# Save the fine-tuned model
output_dir = f"afroXLMR_{LANGUAGE}_sentiment"

print(f"Saving fine-tuned model to {output_dir}...")
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

print("\n✓ Model saved successfully!")
print(f"\nYou can load the model later using:")
print(f"  model = AutoModelForSequenceClassification.from_pretrained('{output_dir}')")
print(f"  tokenizer = AutoTokenizer.from_pretrained('{output_dir}')")

## Step 18: Summary and Key Takeaways

### What We Learned

1. **Fine-tuning Process**
   - Started with a pre-trained model (AfroXLMR)
   - Adapted it to a specific task (sentiment analysis)
   - Used a small, task-specific dataset
   - Much faster than training from scratch!

2. **Transfer Learning Benefits**
   - AfroXLMR already knows African languages
   - We just taught it to classify sentiments
   - Like teaching a multilingual person a new skill!

3. **Evaluation Matters**
   - Multiple metrics give a complete picture
   - Confusion matrix shows where model struggles
   - Important to check per-class performance

### Connection to Lecture
Remember the analogy:
- **Pre-training** (AfroXLMR on web data) = Going to school
- **Fine-tuning** (Our sentiment task) = Medical school specialization

### Model Performance
Your model achieved:
- **Accuracy**: {:.2%}
- **F1-Score**: {:.4f}

This shows the power of transfer learning for African languages!

## Exercise Questions

1. **Understanding Fine-tuning**: Why do we use a small learning rate (2e-5) instead of a large one during fine-tuning?

2. **Language Comparison**: Try fine-tuning on a different African language. How do the results compare? Why might performance differ?

3. **Error Analysis**: Look at the confusion matrix. Which sentiments does the model confuse most? Why?

4. **Data Requirements**: We used hundreds of examples. How many examples would you need for good performance? What's the minimum?

5. **Real-world Application**: How would you deploy this model for real-time sentiment analysis of social media in your chosen language?

6. **Improvement Ideas**: What could you do to improve model performance?
   - More training data?
   - Different hyperparameters?
   - Ensemble methods?
   - Data augmentation?

## Optional: Try Different Languages

Want to compare performance across languages? Just change the `LANGUAGE` variable in Step 3 and re-run the notebook!